In [1]:
from google.colab import drive
drive.mount('/content/drive')
folder = 'drive/MyDrive/Code/Python/Collab/Cellular Automata 2020'

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.colors as cl
from tensorflow.keras.layers import Conv2D
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
import moviepy.editor as mvp
import platform, tqdm
import PIL.Image, PIL.ImageDraw
from IPython.display import Image, clear_output
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3727360/45929032 bytes (8.1%)7831552/45929032 bytes (17.1%)11829248/45929032 bytes (25.8%)15654912/45929032 bytes (34.1%)19570688/45929032 bytes (42.6%)23683072/45929032 bytes (51.6%)27615232/45929032 bytes (60.1%)31735808/45929032 bytes (69.1%)35905536/45929032 bytes (78.2%)39985152/45929032 bytes (87.1%)44105728/45929032 bytes (96.0%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

In [ ]:
JustTesting = True # If True run everything faster

LoadPreviousModelQ = False # if True we load the model, either for further training or just testing
i_step_load = 360000

RunTrainingQ = False # If True we run the training

RebuildDatasetQ = False # if True we rebuild the dataset
MutateTrainingQ = True # if True, during training we mutate the image
MutateTestingQ = True # if True, during testing we mutate the image

i_step_verify = [] #[240000, 500000]
RunTestQ = True # if True, in the end we test an increasing size of inputs

no_channels = '4TimesClasses' # 'SameClasses' '5PlusClasses' '4TimesClasses'

dataset = 'count_digits'
id_run = 'CA_CD_3Classes_Deeper_M{}_MutTrain{}_MutTest{}'.format(no_channels, MutateTrainingQ, MutateTestingQ) # the prefix to all file names which will be use for saving and loading the model

NO_CLASSES = 3
limits_classes = [2, 8] #len of this should be NO_CLASSES-1
limits_c_p = [0, 2, 8, 100]
H, W = 10, 10

TR_EVOLVE = 50 # Number of time steps to let CA evolve for each input during training
TST_EVOLVE = 50 # Number of time steps to let CA evolve for each input during testing

BATCH_SIZE = 64 # number of images per batch


if no_channels == 'SameClasses':
    NO_CHANNELS = NO_CLASSES # number of hidden states of the CA, must be at least NO_CLASSES because there are two outputs
elif no_channels == '4TimesClasses':
    NO_CHANNELS = 4 * NO_CLASSES # number of hidden states of the CA, must be at least NO_CLASSES because there are two outputs
elif no_channels == '5PlusClasses':
    NO_CHANNELS = 5 + NO_CLASSES # number of hidden states of the CA, must be at least NO_CLASSES because there are two outputs
    
if JustTesting:
    TR_NO_ITERATIONS = 500 # number of iterations for the training loop
    export_every = 250 # number of iterations between each model export
    visualise_every = 50 # number of iteration between each model visualisation
else:
    TR_NO_ITERATIONS = 500000 # number of iterations for the training loop
    export_every = 10000 # number of iterations between each model export
    visualise_every = 2000 # number of iteration between each model visualisation

ADD_NOISE = True # if True then the normal update of the CA has noise added

In [ ]:
class CAModel(tf.keras.Model):
    
    def __init__(self, add_noise=ADD_NOISE):
        super().__init__()
        self.add_noise = add_noise

        self.update_state = tf.keras.Sequential([
            Conv2D(80, 3, activation=tf.nn.relu, padding="SAME"),
            Conv2D(120, 1, activation=tf.nn.relu, padding="SAME"),
            Conv2D(NO_CHANNELS, 1, activation=None, padding="SAME"),
        ])
        
        self(tf.zeros([1, H, W, 1 + NO_CHANNELS])) # dummy call to build the model
    
    @tf.function
    def call(self, x):
        '''
        this function updates the CA for one cycle
        x is the current CA state. its shape is (batch_size, H, W, no_channels). 
            batch_size is BATCH_SIZE.
            no_channels is 1 + NO_CHANNELS, 
                where the first is the gray image, 
                the last NO_CLASSES are the classification predictions,
                and the others are there just for fun :)
        '''
        ds = self.update_state(x) # ds will be the state update (of course, we don't want to update the gray image as that is our true input)
        image, state = tf.split(x, [1, NO_CHANNELS], -1)
        if self.add_noise:
            residual_noise = tf.random.normal(tf.shape(ds), mean=0., stddev=0.02)
            ds += residual_noise

        ds *= image 
        state += ds

        return tf.concat([image, state], -1)

    @tf.function
    def initialize(self, images):
        '''
        input: images of size (batch, h, w)
        output: initial CA state full of 0's for positions other than the images. shape (batch, h, w, 1 + channel_n)
        '''
        state = tf.zeros([tf.shape(images)[0], H, W, NO_CHANNELS]) # size (batch, h, w, channel_n) full of zeros
        images = tf.reshape(images, [-1, H, W, 1]) # our images we add an extra dimension
        return tf.concat([images, state], -1) # just concatenating

    @tf.function
    def classify(self, x):
        '''
        The last NO_CLASSES layers are the classification predictions, one channel
        per class.
        '''
        return x[:, :, :, -NO_CLASSES:]
    
    @tf.function
    def mutate(self, x, new_images):
        '''
        This function corrupts the current state of the CA by just changing the gray image
        '''
        image, state = tf.split(x, [1, NO_CHANNELS], -1)
        return tf.concat([new_images, state], -1)

CAModel().update_state.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (1, 10, 10, 80)           9440      
_________________________________________________________________
conv2d_1 (Conv2D)            (1, 10, 10, 120)          9720      
_________________________________________________________________
conv2d_2 (Conv2D)            (1, 10, 10, 12)           1452      
Total params: 20,612
Trainable params: 20,612
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Training utilities
@tf.function
def individual_l2_loss(x, y):
    '''
    x is the current CA state vector. its shape is (batch_size, height, width, no_channels).
    y is the correct label out of NO_CLASSES possibilities. its shape is (batch_size, height, width, NO_CLASSES) (one-hot)
    '''
    t = y - ca.classify(x) # basically we want 1's for the correct and 0s for the incorrect digit. its shape is (batch_size, height, width, NO_CLASSES) (one-hot)
    error_batch = tf.reduce_sum(t ** 2, [1, 2, 3]) / 2
    no_pixels = tf.reduce_sum(y, [1, 2, 3])
    error_normalised_batch = error_batch / no_pixels
    return error_normalised_batch

@tf.function
def batch_l2_loss(x, y, label_vector):
    '''
    x is the current CA state vector. its shape is (batch_size, height, width, no_channels).
    y is the correct label out of 10 possibilities. its shape is (batch_size, height, width, 10) (one-hot)
    returns the mean of the loss function
    '''
    i_l = individual_l2_loss(x, y)
    class_loss = []
    for i in range(NO_CLASSES):
        idx = tf.where((label_vector > (i - 0.5)) & (label_vector < (i + 0.5)))
        gather = tf.gather(i_l, tf.reshape(idx, (-1,)))
        class_loss.append(tf.reduce_mean(gather))
    return tf.reduce_mean(i_l), class_loss

lr = 1e-3 # initial learning rate
lr_sched = tf.keras.optimizers.schedules.PiecewiseConstantDecay([int(TR_NO_ITERATIONS*0.3), int(TR_NO_ITERATIONS*0.7)], [lr, lr*0.1, lr*0.01])
trainer = tf.keras.optimizers.Adam(lr_sched) # use ADAM optimizer with learning rate schedule

loss_log = np.zeros(TR_NO_ITERATIONS) # for plotting of loss function across time
loss_log_classes = np.zeros((TR_NO_ITERATIONS, NO_CLASSES)) # for plotting of loss function across time

def export_model(ca, i, loss_log, loss_log_classes):
    '''
    Saves the models parameters in file name base_fn
    '''
    ca.save_weights(folder + '/CA/' + id_run + '_run_no_{}'.format(i))
    np.savez(folder + '/CA/' + id_run + '_run_no_{}_loss'.format(i), loss_log=loss_log, loss_log_classes=loss_log_classes)

def get_model(i):
    ca = CAModel(add_noise=ADD_NOISE)
    ca.load_weights(folder + '/CA/' + id_run + '_run_no_{}'.format(i))
    res = np.load(folder + '/CA/' + id_run + '_run_no_{}_loss.npz'.format(i))
    loss_log = res['loss_log']
    loss_log_classes = res['loss_log_classes']
    return ca, loss_log, loss_log_classes

In [ ]:
# Visualization utilities

def zoom(img, scale=4):
    '''
    Takes an image array and increases its resolution by simply repeating the values
    '''
    img = np.repeat(img, scale, 0)
    img = np.repeat(img, scale, 1)
    return img

def tile2d(a, w=None):
    a = np.asarray(a)
    if w is None:
        w = int(np.ceil(np.sqrt(len(a))))
    th, tw = a.shape[1:3]
    pad = (w-len(a))%w
    a = np.pad(a, [(0, pad)]+[(0, 0)]*(a.ndim-1), 'constant')
    h = len(a)//w
    a = a.reshape([h, w]+list(a.shape[1:]))
    a = np.rollaxis(a, 2, 1).reshape([th*h, tw*w]+list(a.shape[4:]))
    return a

def np2pil(a):
    # Covert Numpy array of floats into ints
    if a.dtype in [np.float32, np.float64]:
        a = np.uint8(np.clip(a, 0, 1) * 255)
    return PIL.Image.fromarray(a)

def imwrite(f, a, fmt=None):
    # Save numpy array a as image in the disk with filename f
    a = np.asarray(a)
    if isinstance(f, str):
        fmt = f.rsplit('.', 1)[-1].lower()
        if fmt == 'jpg':
            fmt = 'jpeg'
        f = open(f, 'wb')
    np2pil(a).save(f, fmt, quality=95)

def imencode(a, fmt='jpeg'):
    '''
    a is the array/tensor to be encoded
    fmt = fileformat
    '''
    a = np.asarray(a)
    if len(a.shape) == 3 and a.shape[-1] == 4:
        fmt = 'png'
    f = io.BytesIO()
    imwrite(f, a, fmt)
    return f.getvalue()

def imshow(a, fmt='jpeg'):
    '''
    a is the array/tensor to be plotted
    fmt = fileformat
    '''
    display(Image(data=imencode(a, fmt)))


# colors = np.array([cl.to_rgba_array(plt.cm.tab10(i))[0, :3] * 255 for i in range(10)]).astype(int)
# 
color_lookup = tf.constant([
                    [255, 0, 0],
                    [0, 255, 0],
                    [0, 0, 255],
                    [0, 0, 0], # This is the default for digits.
                    [255, 255, 255] # This is the background.
                    ])

def color_labels(x, output_x, dtype=tf.uint8):
    '''
    input x grayscale image: its shape is ([batch_size, ]height, width)
    input output_x: the output of the CA for the image x. its shape seems to be ([batch_size, ]height, width, 10)
    '''
    is_alive = x # 1 if pixel is alive, 0 if background. its shape ([b, ]h, w)
    is_background = 1. - x # 0 if pixel is alive, 1 if background. its shape ([b, ]h, w)
    output_x = output_x * tf.expand_dims(is_alive, -1) # forcibly cancels everything outside of it. y_pic will be 0 for dead pixels. and the previous value for the alive pixels
    
    black_and_bg = tf.fill(list(x.shape) + [2], 0.01) # an array filled with 0.01 of shape ([b, ]h, w, 2). 0.01 is the minimum value for outputs to be considered
    black_and_bg *= tf.stack([is_alive, is_background], -1) # an array filled with 0.01 and 0's depending on alive status. of shape ([b, ]h, w, 2)

    number_or_background =  tf.concat([output_x, black_and_bg], -1) # the voting for the 12 different categories. (10 colors + original grayscale and background. its shape is ([b, ]h, w, 12)
    classified_pixels = tf.argmax(number_or_background, -1) # the result of the vote.  its shape is ([b, ]h, w)
    rgb_pixels = tf.gather(color_lookup, classified_pixels) # And now I suppose this shape is ([b, ]h, w, 3)
    if dtype == tf.uint8:
        return tf.cast(rgb_pixels, tf.uint8)
    else:
        return tf.cast(rgb_pixels, dtype) / 255.

def classify_and_color(ca, x):
    '''
    input ca: the CA class instance with proper architecture and learned weights
    input x: a state of the CA. its shape is (batch_size, height, width, no_channels).
    output: 
    '''
    current_image = x[:, :, :, 0]
    output_ca = ca.classify(x) # x[:, :, :, -NO_CLASSES:] shape = (b, h, w, NO_CLASSES) where we want values 0 if not that number and 1 if that number
    return color_labels(current_image, output_ca, dtype=tf.float32)

def visualize_batch(ca, x0, x, step_i):
    '''
    input ca: the CA class instance with proper architecture and learned weights
    input x0: the initial state in the learning step. its shape is (batch_size, height, width, no_channels).
    input x: the final state. its shape is (batch_size, height, width, no_channels).

    '''
    vis0 = np.hstack(classify_and_color(ca, x0).numpy()) # create horizontally the batch with proper colors
    vis1 = np.hstack(classify_and_color(ca, x).numpy())
    vis = np.vstack([vis0, vis1]) # before and after the steps
    imwrite(folder + '/CA/batches_%04d.jpg'%step_i, vis)
    imshow(vis)

def plot_loss(loss_log, loss_log_classes, save=False):
    plt.figure(figsize=(10, 4))
    plt.title('Loss history (log10)')
    x = loss_log
    plt.plot(np.log10(loss_log), '.', alpha=0.1)
    for i in range(NO_CLASSES):
        if loss_log_classes.shape[0] > 0:
            plt.plot(np.log10(loss_log_classes[:, i]), alpha=0.5, label='{}<pixels<={}'.format(limits_c_p[i], limits_c_p[i+1]), color=color_lookup[i].numpy()/255)
    plt.legend()
    if save:
        plt.savefig(folder + '/CA/log_loss_{}'.format(id_run))
    plt.show()

In [ ]:
# Prepare the dataset
size_ds = 10000

def get_next_pixel(base_pixel, positions_colored):
    x, y = base_pixel[0], base_pixel[1]
    keep_going = 1
    j = 0
    while keep_going and j < 10:
        j += 1
        x_or_y = np.random.rand() < 0.5
        pos_or_neg = np.random.rand() < 0.5
        if x_or_y:
            if pos_or_neg:
                dx = 1
                dy = 0
            else:
                dx = -1
                dy = 0
        else:
            if pos_or_neg:
                dx = 0
                dy = 1
            else:
                dx = 0
                dy = -1
        nx = min(max(0, x + dx), 9)
        ny = min(max(0, y + dy), 9)
        test_pixel = [nx, ny]
        if test_pixel in positions_colored:
            pass
        else:
            return 1, test_pixel
    return 0, [0, 0]

BuildDS = False
if RebuildDatasetQ:
    BuildDS = True
else:
    try:
        res = np.load(folder + '/CA/{}_{}.npz'.format(dataset, size_ds))
        x_train = res['x_train']
        y_train = res['y_train']
        x_test = res['x_test']
        y_test = res['y_test']
    except:
        BuildDS = True
if BuildDS:
    X = np.zeros((size_ds, H, W), np.float32)
    Y = np.zeros((size_ds), np.float32)
    
    for i in range(size_ds):
        no_pixels = np.random.randint(20)
        pixels_placed = 1
        first_pixel = [4, 4]
        positions_colored = [first_pixel]
        X[i, first_pixel[0], first_pixel[1]] = 1
        for j in range(no_pixels - 1):
            base_pixel = positions_colored[np.random.choice(len(positions_colored))]
            sucess, next_pixel = get_next_pixel(base_pixel, positions_colored)
            if sucess:
                pixels_placed += 1
                positions_colored.append(next_pixel)
                X[i, next_pixel[0], next_pixel[1]] = 1
        Y[i] = pixels_placed
        x_train, x_test = np.split(X, [int(size_ds*0.8)])
        y_train, y_test = np.split(Y, [int(size_ds*0.8)])
    

    np.savez(folder + '/CA/{}_{}.npz'.format(dataset, size_ds), x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)
    
def class_indice_f(no_pixels):
    indices = np.where(no_pixels <= np.array(limits_classes))[0]
    if indices.shape[0] == 0:
        return NO_CLASSES-1
    else:
        return indices[0]

def to_classes_dim_label(x, y):
    '''
    input x shape is (no_images, height, width)
    input y shape is (no_images,)
    output: y_res (no_images, height, width, NO_CLASSES) y_res[b, h, w, i] = 1 if the image b is digit i, and only at the positions h, w where it is alive
    '''
    y_res = np.zeros(list(x.shape) + [NO_CLASSES])
    y_expanded = np.broadcast_to(y, x.T.shape).T # broadcast y to match x shape:
    for i in range(x.shape[0]):
        class_indice = class_indice_f(y[i])
        y_res[i, :, :, class_indice] = x[i, :, :]
    return y_res.astype(np.float32)


y_train_pic = to_classes_dim_label(x_train, y_train)
y_label_train = np.zeros(y_train.shape[0])
for i in range(y_train.shape[0]):
    y_label_train[i] = class_indice_f(y_train[i])
# y_test_pic = to_classes_dim_label(x_test, y_test)

In [ ]:
if LoadPreviousModelQ:
    ca, loss_log, loss_log_classes = get_model(i_step_load)
    ITER = i_step_load
else:
    ca = CAModel()
    ITER = 0

In [ ]:
# Training happens here
if RunTrainingQ:
    # Training Step
    @tf.function
    def train_step(x, y, y_label, MutateTrainingQ=False):
        '''
        x is the current CA state. its shape is (batch_size, height, width, no_channels).
        y is the correct label out of 10 possibilities. its shape is (batch_size, ?)
        '''
        iter_n = max(2, np.random.randint(TR_EVOLVE)) # Number of initial iterations of the CA for each training step
        with tf.GradientTape() as g: # GradientTape does automatic differentiation on the learnable_parameters of our model
            for i_iter in tf.range(iter_n): # Basically let time evolve
                x = ca(x) # update the CA according to call method? ca(x) = ca.call(x)?
            loss_b, c_l_b = batch_l2_loss(x, y, y_label) # compute the scalar loss
        grads = g.gradient(loss_b, ca.weights) # Gradient Tape and Keras doing its magic
        grads = [g/(tf.norm(g)+1e-8) for g in grads] # Normalising the gradients uh?
        trainer.apply_gradients(zip(grads, ca.weights)) # Keras and ADAM magic 
        
        if MutateTrainingQ:
            c_i = x[:, :, :, 0]
            x = ca.mutate(x, tf.expand_dims(tf.random.shuffle(c_i), -1))
        
        iter_n = TR_EVOLVE - iter_n # Number of iterations of the CA for each training step
        with tf.GradientTape() as g: # GradientTape does automatic differentiation on the learnable_parameters of our model
            for i_iter in tf.range(iter_n): # Basically let time evolve
                x = ca(x) # update the CA according to call method? ca(x) = ca.call(x)?
            loss_a, c_l_a = batch_l2_loss(x, y, y_label) # compute the scalar loss
        grads = g.gradient(loss_a, ca.weights) # Gradient Tape and Keras doing its magic
        grads = [g/(tf.norm(g)+1e-8) for g in grads] # Normalising the gradients uh?
        trainer.apply_gradients(zip(grads, ca.weights)) # Keras and ADAM magic 
        return x, loss_b + loss_a, [c_l_b[i_list] + c_l_a[i_list] for i_list in range(NO_CLASSES)]

    # Training Loop
    for i in range(ITER, TR_NO_ITERATIONS):

        b_idx = np.random.randint(0, x_train.shape[0] - 1, size=BATCH_SIZE)
        x0 = ca.initialize(x_train[b_idx])
        y0 = y_train_pic[b_idx]
        y0_label = y_label_train[b_idx]
        y0_label = tf.convert_to_tensor(y0_label)        

        x, loss, c_l = train_step(x0, y0, y0_label)

        loss_log[i] = loss.numpy()
        loss_log_classes[i, :] = [k.numpy() for k in c_l]

        if i % visualise_every == 0:
            clear_output()
            plot_loss(loss_log[:i], loss_log_classes[:i, :], True)
        if i % export_every == 0:
            export_model(ca, i, loss_log, loss_log_classes)

        print('\r step: {}, log10(loss): {}, log10(loss)[classes]: {}'.format(i + 1, np.log10(loss), np.log10(c_l)), end='')
    export_model(ca, TR_NO_ITERATIONS, loss_log, loss_log_classes)

In [ ]:
class VideoWriter:
    def __init__(self, filename, fps=30.0, **kw):
        self.writer = None
        self.params = dict(filename=filename, fps=fps, **kw)

    def add(self, img):
        img = np.asarray(img)
        if self.writer is None:
            h, w = img.shape[:2]
            self.writer = FFMPEG_VideoWriter(size=(w, h), **self.params)
        if img.dtype in [np.float32, np.float64]:
            img = np.uint8(img.clip(0, 1)*255)
        if len(img.shape) == 2:
            img = np.repeat(img[..., None], 3, -1)
        if len(img.shape) == 3 and img.shape[-1] == 4:
            img = img[..., :3] * img[..., 3, None]
        self.writer.write_frame(img)

    def close(self):
        if self.writer:
            self.writer.close()

    def __enter__(self):
        return self

    def __exit__(self, *kw):
        self.close()

def make_run_videos(ca, eval_bs, i_step, seed=1):
    np.random.seed(seed)
    new_idx = np.random.randint(0, x_test.shape[0] - 1, size=eval_bs)
    x = ca.initialize(x_test[new_idx])
    with VideoWriter(folder + '/CA/Movie_model_{}_{}.mp4'.format(id_run, i_step)) as vid:
        for i in tqdm.trange(-1, TST_EVOLVE):
            if MutateTestingQ and i == int(TST_EVOLVE / 2):
                c_i = x[:, :, :, 0]
                x = ca.mutate(x, tf.expand_dims(tf.random.shuffle(c_i), -1))
#             true_images = tf.expand_dims(0.5 - 0 * x[:, :, :, 0], -1)
#             alphas = zoom(tile2d(true_images), scale=4)
#             print(alphas)
            if i == -1:
                image = zoom(tile2d(classify_and_color(ca, x)), scale=4)
            else:
                x = ca(x)
                image = zoom(tile2d(classify_and_color(ca, x)), scale=4)
#             new_image = np.concatenate((image, alphas), axis=-1)
#             print(new_image)
#             im = np.uint8(new_image*255)
            im = np.uint8(image*255)
            im = PIL.Image.fromarray(im)
            draw = PIL.ImageDraw.Draw(im)
            vid.add(np.uint8(im))

In [ ]:
ca

(<__main__.CAModel at 0x7f3647f06e80>,
 array([57.41350555,  7.63882589,  3.03335905, ...,  0.        ,
         0.        ,  0.        ]),
 array([[ 2.22993231, 37.75796127, 92.16165924],
        [ 0.90170944,  6.34681988, 10.06003284],
        [ 0.88766009,  2.69141102,  3.84651518],
        ...,
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]]))

In [ ]:
eval_bs = 5 ** 2 # number of samples in a batch to evaluate
for i_step_v in i_step_verify:
    ca, loss_log, loss_log_classes = get_model(i_step_v)
    make_run_videos(ca, eval_bs, i_step_v)

  0%|          | 1/201 [00:00<00:32,  6.24it/s]

100%|██████████| 201/201 [00:00<00:00, 231.42it/s]


In [ ]:
if RunTestQ:
    def add_pixel(old_image):
        new_image = old_image.copy()
        indices = np.asarray(np.where(old_image == 1)).T
        success = 0
        k = 0
        while (not success) or (k > 100):
            k += 1
            idx = np.random.randint(indices.shape[0])
            x, y = indices[idx, :]
            x_or_y = np.random.rand() < 0.5
            pos_or_neg = np.random.rand() < 0.5
            if x_or_y:
                if pos_or_neg:
                    dx = 1
                    dy = 0
                else:
                    dx = -1
                    dy = 0
            else:
                if pos_or_neg:
                    dx = 0
                    dy = 1
                else:
                    dx = 0
                    dy = -1
            nx = min(max(0, x + dx), 9)
            ny = min(max(0, y + dy), 9)
            if old_image[nx, ny] < 0.5:
                new_image[nx, ny] = 1
                success = 1
        return new_image

    images = np.zeros((20, 25, H, W), dtype=np.float32)
    for i_img in range(25):
        images[0, i_img, int(H / 2), int(W / 2)] = 1
    for j in range(1, 20):
        for i_img in range(25):
            images[j, i_img, :, :] = add_pixel(images[j - 1, i_img, :, :])
    images = tf.constant(images)
    
    for i_step_v in i_step_verify:
        ca, loss_log, loss_log_classes = get_model(i_step_v)
        x = ca.initialize(images[0, :, :, :])
        with VideoWriter(folder + '/CA/Movie_test_increase_{}_{}.mp4'.format(id_run, i_step_v)) as vid:
            for j in range(20):
                x = ca.mutate(x, tf.expand_dims(images[j, :, :, :], -1))
                for i in tqdm.trange(TR_EVOLVE):
                    x = ca(x)
                    image = zoom(tile2d(classify_and_color(ca, x)), scale=4)
                    im = np.uint8(image*255)
                    im = PIL.Image.fromarray(im)
                    draw = PIL.ImageDraw.Draw(im)
                    vid.add(np.uint8(im))
        x = ca.initialize(images[0, :, :, :])
        with VideoWriter(folder + '/CA/Movie_test_decrease_{}_{}.mp4'.format(id_run, i_step_v)) as vid:
            for j in range(20):
                x = ca.mutate(x, tf.expand_dims(images[19 - j, :, :, :], -1))
                for i in tqdm.trange(TR_EVOLVE):
                    x = ca(x)
                    image = zoom(tile2d(classify_and_color(ca, x)), scale=4)
                    im = np.uint8(image*255)
                    im = PIL.Image.fromarray(im)
                    draw = PIL.ImageDraw.Draw(im)
                    vid.add(np.uint8(im))

100%|██████████| 20/20 [00:00<00:00, 295.13it/s]
